# Llama model pre-training on Intel Gaudi

<a id="try-anyscale-quickstart-intel_gaudi-llama_pretrain" href="https://console.anyscale.com/register/ha?render_flow=ray&utm_source=ray_docs&utm_medium=docs&utm_campaign=intel_gaudi-llama_pretrain">
    <img src="../../../_static/img/run-on-anyscale.svg" alt="try-anyscale-quickstart">
</a>
<br></br>

In this Jupyter notebook, we will pre-train a [huggyllama/llama-7b](https://huggingface.co/huggyllama/llama-7b) model by using Intel Gaudi accelerators.

We will use PyTorch for model training and Ray for distributed training.

[Intel Gaudi AI Processors (HPUs)](https://habana.ai) are AI hardware accelerators designed by Habana Labs. For more information, see [Gaudi Architecture](https://docs.habana.ai/en/latest/Gaudi_Overview/index.html) and [Gaudi Developer Docs](https://developer.habana.ai/).

Basic features for this pre-training example are:
- Running on HPUs, support three execution mode: ["lazy", "eager", "eager.compile"](https://docs.habana.ai/en/latest/PyTorch/Reference/PyTorch_Gaudi_Theory_of_Operations.html).
- Pre-training llama model use configuration [huggyllama/llama-7b](https://huggingface.co/huggyllama/llama-7b)
- [`GaudiTrainer`](https://github.com/huggingface/optimum-habana/blob/main/optimum/habana/transformers/trainer.py) based training.
- DeepSpeed based pre-training.
- Ray based resource scheduling and management.

## Prepare environment
This example run on single node with 4 HPUs.

We recommend using a prebuilt container to run these examples. To run a container, you need Docker. See [Install Docker Engine](https://docs.docker.com/engine/install/) for installation instructions.

Next, follow [Run Using Containers](https://docs.habana.ai/en/latest/Installation_Guide/Bare_Metal_Fresh_OS.html?highlight=installer#run-using-containers) to install the Habana drivers and container runtime.

### Get docker image
``` bash
# more available docker image can be found here: https://vault.habana.ai/ui/native/gaudi-docker
docker pull vault.habana.ai/gaudi-docker/1.22.1/ubuntu24.04/habanalabs/pytorch-installer-2.7.1:latest
```
### Run docker image
``` bash
docker run -it --runtime=habana -e HABANA_VISIBLE_DEVICES=all -e OMPI_MCA_btl_vader_single_copy_mechanism=none --cap-add=sys_nice --net=host --ipc=host vault.habana.ai/gaudi-docker/1.22.1/ubuntu24.04/habanalabs/pytorch-installer-2.7.1:latest
# maybe should mapping your workspace volumns
```
### Install dependency
``` bash
# "optimum-habana>1.11.1" if execution mode "eager" or "eager.compile" 
# "ray>=2.20.0"
pip install ray[train] notebook transformers datasets evaluate peft==0.17.1 accelerate scikit-learn optimum-habana

# install deepspeed
pip install git+https://github.com/HabanaAI/DeepSpeed.git@1.22.0

# this notebook verfied with packages' version:
# transformers==4.57.1
# datasets==4.4.1
# evaluate==0.4.6
# peft==0.17.1
# accelerate==1.11.0
# scikit-learn==1.7.2
# optimum-habana==1.19.1

# deepspeed==0.16.1+hpu.synapse.v1.22.0
```

## Import necessary libraries

In [ ]:
#!/usr/bin/env python

import os
from typing import Any, Dict
from torch.utils.data import DataLoader

import transformers
from itertools import chain
from datasets import load_dataset
from transformers import default_data_collator
from transformers.testing_utils import CaptureLogger
from optimum.habana import GaudiConfig, GaudiTrainer, GaudiTrainingArguments
from optimum.habana.utils import set_seed

## Build datasets

Download and load dataset from huggingface.co

In [ ]:
def load_datasets(config):
    dataset_name = config["name"] 
    dataset_config_name = config["config_name"]

    # Downloading and loading a dataset from the hub.
    raw_datasets = load_dataset(
        dataset_name,
        dataset_config_name,
        cache_dir=None,
        token=None,
        streaming=False,
    )
    if "validation" not in raw_datasets.keys():
        raw_datasets["validation"] = load_dataset(
            dataset_name,
            dataset_config_name,
            split=f"train[:{data_args.validation_split_percentage}%]",
            cache_dir=None,
            token=None,
            streaming=False,
        )
        raw_datasets["train"] = load_dataset(
            dataset_name,
            dataset_config_name,
            split=f"train[{data_args.validation_split_percentage}%:]",
            cache_dir=None,
            token=None,
            streaming=False,
        )

    return raw_datasets

## Load tokenizer

Download vocabulary from huggingface.co.

In [ ]:
def load_tokenizer(config):
    name = config["name"]
    tokenizer_kwargs = {
        "cache_dir": None,
        "use_fast": True,
        "revision": "main",
        "token": None,
        "trust_remote_code": False,
    }
    return transformers.AutoTokenizer.from_pretrained(name, **tokenizer_kwargs)

## Tokenize dataset

tokenize word to token ids.

In [ ]:
def tokenize_dataset(datasets, tokenizer):
    column_names = list(datasets["train"].features)
    text_column_name = "text" if "text" in column_names else column_names[0]

    tok_logger = transformers.utils.logging.get_logger("transformers.tokenization_utils_base")

    def tokenize_function(examples):
        with CaptureLogger(tok_logger) as cl:
            output = tokenizer(examples[text_column_name])
        # clm input could be much much longer than block_size
        if "Token indices sequence length is longer than the" in cl.out:
            tok_logger.warning(
                "^^^^^^^^^^^^^^^^ Please ignore the warning above - this long input will be chunked into smaller bits"
                " before being passed to the model."
            )
        return output

    tokenized_datasets = datasets.map(
        tokenize_function,
        batched=True,
        num_proc=None,
        remove_columns=column_names,
        load_from_cache_file=True,
        desc="Running tokenizer on dataset",
    )

    return tokenized_datasets

## Group dataset

This preprocssing will concatenate all texts from our dataset and generate chunks of block_size, and will pre-train model much faster.

In [ ]:
def group_dataset(config, datasets, tokenizer):
    config_name = config["name"]
    auto_config = transformers.AutoConfig.from_pretrained(config_name)
    max_pos_embeddings = auto_config.max_position_embeddings
    block_size = tokenizer.model_max_length
    if block_size > max_pos_embeddings:
        print(
            f"The tokenizer picked seems to have a very large `model_max_length` ({tokenizer.model_max_length}). "
            f"Using block_size={min(1024, max_pos_embeddings)} instead. You can change that default value by passing --block_size xxx."
        )
        if max_pos_embeddings > 0:
            block_size = min(1024, max_pos_embeddings)
        else:
            block_size = 1024

    # Main data processing function that will concatenate all texts from our dataset and generate chunks of block_size.
    def group_texts(examples):
        # Concatenate all texts.
        concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        # We drop the small remainder, and if the total_length < block_size  we exclude this batch and return an empty dict.
        # We could add padding if the model supported it instead of this drop, you can customize this part to your needs.
        total_length = (total_length // block_size) * block_size
        # Split by chunks of max_len.
        result = {
            k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
        result["labels"] = result["input_ids"].copy()
        return result

    lm_datasets = datasets.map(
        group_texts,
        batched=True,
        num_proc=None,
        load_from_cache_file=True,
        desc=f"Grouping texts in chunks of {block_size}",
    )
    return lm_datasets

## Load model

Download and load pre-configed model from huggingface.co, the detail model configurations in config.json

In [ ]:
def load_model(config):
    name = config["name"]
    model_config = config.get("config", {})
    auto_config = transformers.AutoConfig.from_pretrained(
        pretrained_model_name_or_path=name, **model_config
    )
    model = transformers.AutoModelForCausalLM.from_config(auto_config, trust_remote_code=False)

    return model

## Prepare trainer

Instance Trainer with `model`, `gaudi_config`, `training_args`, `tokenizer`

No evaluation dataset passed, just training.

In [ ]:
def get_trainer(training_args, datasets, tokenizer, model):
    gaudi_config = GaudiConfig.from_pretrained(
        training_args.gaudi_config_name, revision="main",
    )

    trainer = GaudiTrainer(
        model=model,
        gaudi_config=gaudi_config,
        args=training_args,
        train_dataset=datasets["train"],
        eval_dataset=None,
        tokenizer=tokenizer,
        data_collator=default_data_collator,
    )
    return trainer

## Training Function

This function will be executed by each worker during training, with following steps:
- prepare GaudiTrainingArguments object.
- load datasets from huggingface.co.
- load pre-configed tokenizer from huggingface.co.
- tokenize dataset with loaded model tokenizer.
- concatenate all texts from our dataset and generate chunks of block_size.
- instance object of `GaudiTrainer` with training_args, datasets, tokenizer, and model.
- call `train` of trainer.
- save model.

In [ ]:
def pretrain_llama(config: Dict[str, Any]):

    training_args = GaudiTrainingArguments(**config["training_args"])
    set_seed(training_args.seed)

    raw_datasets = load_datasets(config["datasets"])

    tokenizer = load_tokenizer(config["tokenizer"])

    tokenized_datasets = tokenize_dataset(raw_datasets, tokenizer)

    tokenized_datasets = group_dataset(config["model"], tokenized_datasets, tokenizer)

    model = load_model(config["model"])

    trainer = get_trainer(training_args, tokenized_datasets, tokenizer, model)

    result = trainer.train()
    trainer.save_model()
    print(result)

## Main Training Function

The `main` function sets up the distributed training environment using Ray and starts the training process. To enable training using HPU, we only need to make the following changes:
- Set the exectuion mode for training, supported execution mode are:

    - "lazy": Deferred execution of graphs, comprising of ops delivered from script op by op similar to Eager mode. It gives the Eager mode experience with performance on Gaudi. Unlike Eager Mode with torch.compile, graph is analyzed in each iteration leading to a higher CPU usage.
    - "eager": Op-by-op execution as defined in standard PyTorch Eager mode scripts.
    - "eager.compile": Eager mode extended with `torch.compile` - Similar to Eager mode but extended with wrapping complete or part of model (such as a function) into a graph. Parts that are not wrapped are executed eagerly.

    More detail theory can be found [here](https://docs.habana.ai/en/latest/PyTorch/Reference/PyTorch_Gaudi_Theory_of_Operations.html), and detail performance results can be found [here](https://developer.habana.ai/get-started/habana-models-performance/)
- Require an HPU for each worker in ScalingConfig
- Set backend to `hccl` in TorchConfig

In [ ]:
def main(num_workers, execution_mode):
    import ray
    from ray.train import ScalingConfig
    from ray.train.torch import TorchTrainer, TorchConfig

    pretrain_config = {
        "datasets": {
            "name": "wikitext",
            "config_name": "wikitext-2-raw-v1",
        },
        "tokenizer": {
            "name": "huggyllama/llama-7b",
            "config": {}
        },
        "model": {
            "name": "huggyllama/llama-7b",
            "config": {
                "torch_dtype": "bfloat16",
            },
        },
        "training_args": {
            "per_device_train_batch_size": 1,
            "do_train": True,
            "save_strategy": "no",
            "output_dir": "/tmp/ray/pretrain-llama-2",
            "logging_steps": 1,
            "gaudi_config_name": "Habana/llama",
            "use_habana": True,
            "throughput_warmup_steps": 3,
            "use_lazy_mode": True,
            "overwrite_output_dir": True,
            "seed": 42,
            "bf16": True,
            "report_to":'tensorboard',
            "deepspeed": {
                "steps_per_print": 64,
                "train_batch_size": "auto",
                "train_micro_batch_size_per_gpu": "auto",
                "gradient_accumulation_steps": "auto",
                "bf16": {
                    "enabled": True
                },
                "gradient_clipping": 1.0,
                "zero_optimization": {
                    "stage": 3,
                    "overlap_comm": False,
                    "reduce_scatter": False,
                    "contiguous_gradients": False,
                    "stage3_gather_16bit_weights_on_model_save": True
                }
            },
        },
    }

    # if execution mode is eager with compile, must spcified with a compile backend
    if execution_mode == "eager.compile":
        pretrain_config["training_args"].update({"torch_compile_backend": "hpu_backend"})

    scaling_config = ScalingConfig(num_workers=num_workers,
                                   use_gpu=False,
                                   resources_per_worker={"CPU": 1, "HPU": 1})

    # Set backend to hccl in TorchConfig
    torch_config = TorchConfig(backend="hccl")

    # Workaround https://github.com/ray-project/ray/issues/45302 by explictly setting HPU resource
    ray.init(resources={"HPU": 8})

    # Initialize a Ray TorchTrainer
    trainer = TorchTrainer(
        train_loop_per_worker=pretrain_llama,
        train_loop_config=pretrain_config,
        torch_config=torch_config,
        scaling_config=scaling_config
    )

    result = trainer.fit()
    print(result)

## Start Training

Finally, we call the `main` function to start the pre-training process.

Before calling `main` function, you must set some environment variables.

1. The visiable devices. Environment variable `HABANA_VISIBLE_DEVICES` and `HABANA_VISIBLE_MODULES` are used to control the HPU device visiable to application, you must set this two environment variable properly. For more detail usage of `HABANA_VISIBLE_DEVICES`, `HABANA_VISIBLE_MODULES`, please visit [here](https://docs.habana.ai/en/latest/PyTorch/Reference/PT_Multiple_Tenants_on_HPU/Multiple_Dockers_each_with_Single_Workload.html)

2. The execution mode. Different execution mode has different runtime performance. The default execution mode is lazy mode.

In [ ]:
# set some environment variables
os.environ["RAY_EXPERIMENTAL_NOSET_HABANA_VISIBLE_MODULES"] = "0"
# if using RAY_EXPERIMENTAL_NOSET_HABANA_VISIBLE_MODULES env var
# you must set HABANA_VISIBLE_MODULES, such as
# os.environ["HABANA_VISIBLE_MODULES"] = "0,1,2,3"

# execution_mode are ["lazy", "eager", "eager.compile"]
execution_mode = "lazy"
os.environ["PT_HPU_LAZY_MODE"] = "1" if execution_mode == "lazy" else "0"

main(num_workers=4, execution_mode=execution_mode)

## Possible outputs

``` text

...

(RayTrainWorker pid=17173) Setting up process group for: env:// [rank=0, world_size=4]
(RayTrainWorker pid=17173) [2025-11-17 23:11:35,362] [INFO] [real_accelerator.py:225:get_accelerator] Setting ds_accelerator to hpu (auto detect)
(RayTrainWorker pid=17176) /usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
(RayTrainWorker pid=17176)   warnings.warn(
(RayTrainWorker pid=17176) Initializing conditional components...
(RayTrainWorker pid=17176) Using HPU fused kernel for apply_rotary_pos_emb
(RayTrainWorker pid=17176) Using HPU fused kernel for RMSNorm
(RayTrainWorker pid=17176) Using HPU fused kernel for apply_rotary_pos_emb
(RayTrainWorker pid=17176) Using HPU fused kernel for RMSNorm
(TrainController pid=16640) Started training worker group of size 4: 
(TrainController pid=16640) - (ip=100.83.67.100, pid=17173) world_rank=0, local_rank=0, node_rank=0
(TrainController pid=16640) - (ip=100.83.67.100, pid=17175) world_rank=1, local_rank=1, node_rank=0
(TrainController pid=16640) - (ip=100.83.67.100, pid=17174) world_rank=2, local_rank=2, node_rank=0
(TrainController pid=16640) - (ip=100.83.67.100, pid=17176) world_rank=3, local_rank=3, node_rank=0

...

(RayTrainWorker pid=17173) ============================= HPU PT BRIDGE CONFIGURATION ON RANK = 0 ============= 
(RayTrainWorker pid=17173)  PT_HPU_LAZY_MODE = 1
(RayTrainWorker pid=17173)  PT_HPU_RECIPE_CACHE_CONFIG = ,false,1024,false
(RayTrainWorker pid=17173)  PT_HPU_MAX_COMPOUND_OP_SIZE = 9223372036854775807
(RayTrainWorker pid=17173)  PT_HPU_LAZY_ACC_PAR_MODE = 0
(RayTrainWorker pid=17173)  PT_HPU_ENABLE_REFINE_DYNAMIC_SHAPES = 0
(RayTrainWorker pid=17173)  PT_HPU_EAGER_PIPELINE_ENABLE = 1
(RayTrainWorker pid=17173)  PT_HPU_EAGER_COLLECTIVE_PIPELINE_ENABLE = 1
(RayTrainWorker pid=17173)  PT_HPU_ENABLE_LAZY_COLLECTIVES = 0
(RayTrainWorker pid=17173) ---------------------------: System Configuration :---------------------------
(RayTrainWorker pid=17173) Num CPU Cores : 160
(RayTrainWorker pid=17173) CPU RAM       : 1007 GB
(RayTrainWorker pid=17173) ------------------------------------------------------------------------------

...

(RayTrainWorker pid=17173) {'loss': 11.1667, 'grad_norm': 15.475007057189941, 'learning_rate': 5e-05, 'epoch': 0.0, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(55.38), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 11.0627, 'grad_norm': 15.35694694519043, 'learning_rate': 4.9951690821256045e-05, 'epoch': 0.01, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(73.24), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 10.9507, 'grad_norm': 15.467569351196289, 'learning_rate': 4.990338164251208e-05, 'epoch': 0.01, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.72), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 10.6487, 'grad_norm': 14.865588188171387, 'learning_rate': 4.985507246376812e-05, 'epoch': 0.01, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.75), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 10.1784, 'grad_norm': 13.968029022216797, 'learning_rate': 4.980676328502415e-05, 'epoch': 0.01, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.75), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 9.9406, 'grad_norm': 16.147335052490234, 'learning_rate': 4.9758454106280194e-05, 'epoch': 0.02, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 9.8046, 'grad_norm': 18.3155574798584, 'learning_rate': 4.9710144927536237e-05, 'epoch': 0.02, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 9.6798, 'grad_norm': 13.360419273376465, 'learning_rate': 4.966183574879227e-05, 'epoch': 0.02, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 9.4443, 'grad_norm': 10.956599235534668, 'learning_rate': 4.9613526570048315e-05, 'epoch': 0.03, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 9.3399, 'grad_norm': 10.75390911102295, 'learning_rate': 4.956521739130435e-05, 'epoch': 0.03, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 8.6311, 'grad_norm': 9.340704917907715, 'learning_rate': 4.9516908212560386e-05, 'epoch': 0.03, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 10.0624, 'grad_norm': 49.417579650878906, 'learning_rate': 4.946859903381643e-05, 'epoch': 0.03, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 10.9374, 'grad_norm': 66.55072021484375, 'learning_rate': 4.9420289855072464e-05, 'epoch': 0.04, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 8.766, 'grad_norm': 13.2837495803833, 'learning_rate': 4.9371980676328506e-05, 'epoch': 0.04, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 8.8053, 'grad_norm': 26.362295150756836, 'learning_rate': 4.932367149758454e-05, 'epoch': 0.04, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 8.6358, 'grad_norm': 7.576889514923096, 'learning_rate': 4.9275362318840584e-05, 'epoch': 0.05, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 8.1603, 'grad_norm': 6.979973316192627, 'learning_rate': 4.922705314009662e-05, 'epoch': 0.05, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 8.473, 'grad_norm': 23.650888442993164, 'learning_rate': 4.9178743961352656e-05, 'epoch': 0.05, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 8.3837, 'grad_norm': 20.919010162353516, 'learning_rate': 4.91304347826087e-05, 'epoch': 0.06, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 8.5428, 'grad_norm': 20.735576629638672, 'learning_rate': 4.9082125603864734e-05, 'epoch': 0.06, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 8.1541, 'grad_norm': 8.866345405578613, 'learning_rate': 4.9033816425120776e-05, 'epoch': 0.06, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 7.883, 'grad_norm': 5.573055267333984, 'learning_rate': 4.898550724637682e-05, 'epoch': 0.06, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 7.7241, 'grad_norm': 8.445459365844727, 'learning_rate': 4.893719806763285e-05, 'epoch': 0.07, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 7.7785, 'grad_norm': 15.916473388671875, 'learning_rate': 4.888888888888889e-05, 'epoch': 0.07, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 7.6401, 'grad_norm': 5.650521755218506, 'learning_rate': 4.884057971014493e-05, 'epoch': 0.07, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 7.6179, 'grad_norm': 7.968504428863525, 'learning_rate': 4.879227053140097e-05, 'epoch': 0.08, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 7.6425, 'grad_norm': 4.692033290863037, 'learning_rate': 4.874396135265701e-05, 'epoch': 0.08, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 7.2658, 'grad_norm': 4.123305797576904, 'learning_rate': 4.8695652173913046e-05, 'epoch': 0.08, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 7.2754, 'grad_norm': 5.893979072570801, 'learning_rate': 4.864734299516908e-05, 'epoch': 0.08, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 7.1253, 'grad_norm': 7.051833152770996, 'learning_rate': 4.8599033816425124e-05, 'epoch': 0.09, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}
(RayTrainWorker pid=17173) {'loss': 7.4429, 'grad_norm': 6.333181381225586, 'learning_rate': 4.855072463768116e-05, 'epoch': 0.09, 'memory_allocated (GB)': np.float64(28.87), 'max_memory_allocated (GB)': np.float64(76.77), 'total_memory_available (GB)': np.float64(94.62)}


...
```